Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [24]:
from collections import namedtuple, deque
from heapq import heappush, heappop
import numpy as np
from tqdm import tqdm
from random import choice

### Initialization

In [25]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])
action_cost = 1

def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal = state
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 223530.38it/s]


array([[21, 15,  0, 11,  4],
       [24, 19,  3, 12,  8],
       [20, 23,  2, 18,  7],
       [16, 10, 14,  9,  1],
       [ 6,  5, 13, 22, 17]])

### A* problem solving

In [ ]:
# Heuristics with distance to goal position
def heuristics(state: np.ndarray) -> int:
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Do not consider the position of the 0
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)  # Target position (all in order 1 then 2 then 3...)
                distance += abs(x - target_x) + abs(y - target_y) # Manhattan method to calculate distance
    return distance

# A* implementation
def a_star(initial_state: np.ndarray, goal_state: np.ndarray):
    # Priority queue (min-heap)
    frontier = []
    heappush(frontier, (0, initial_state.tobytes(), []))  # Add initial state as (f, state_bytes, path)
    
    # Visited states as a set
    visited = set()
    
    # Progress bar: total is unknown initially, so we use `total=None`
    progress = tqdm(total=0, desc="Processing States", position=0, leave=True, dynamic_ncols=True)

    while frontier:
        # Update progress bar for each state processed
        progress.total = len(frontier) + len(visited)  # Approximate total states explored so far
        progress.update(1)  # Increment progress by 1

        # Pop the state with the smallest cost from the priority queue
        f, current_bytes, path = heappop(frontier)

        # Convert the byte string back to a NumPy array
        current_state = np.frombuffer(current_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # If the current state matches the goal, return the path and its cost
        if np.array_equal(current_state, goal_state):
            progress.close()
            total_cost = len(path) * action_cost
            return path, total_cost

        # Add the current state to the visited set
        visited.add(current_bytes)

        # Explore all possible actions
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes in visited:
                continue

            # Calculate cost
            g = len(path) + action_cost  # Actual cost to reach this state
            h = heuristics(new_state)  # Heuristic cost to goal
            f = g + h  # Total estimated cost

            # Add the new state to the priority queue
            heappush(frontier, (f, new_state_bytes, path + [act]))

    # If no solution is found
    progress.close()
    return None, None


# Résolution du puzzle
solution = a_star(state, goal)
if solution is not None:
    print(f"Puzzle résolu en {len(solution)} mouvements.")
    for step, act in enumerate(solution, 1):
        print(f"Mouvement {step}: échange {act.pos1} avec {act.pos2}")
else:
    print("Aucune solution trouvée.")

Processing States:  49%|████▊     | 9862500/20292318 [12:13<07:30, 23161.11it/s] 